In [102]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
PI = 3.1415926

def load_image(path):
    return cv2.imread(path, 0)

def gamma(img):
    height, width = img.shape
    for i in range(0, height, 1):
        for j in range(0, width, 1):
            img[i,j] = img[i,j] ** 0.5
            
def sobel_gradient(img):
    height, width = img.shape
    gx = cv2.Sobel(img, cv2.CV_16S, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_16S, 0, 1)
    grad_array = np.ndarray((height, width))
    angle_array = np.ndarray((height, width))
    for i in range(0, height, 1):
        for j in range(0, width, 1):
            grad_array[i,j] = math.sqrt(gx[i,j]**2 + gy[i,j]**2)
            if gy[i,j] == 0:
                angle_array[i,j] = 0
            else:
                angle_array[i,j] = np.arctan(gx[i,j]/gy[i,j])
    return grad_array, angle_array
    
def single_cell_hist(grad_cell, angle_cell):
    cell_list = [0] * 6
    ang = 0
    height, width = angle_cell.shape
    for i in range(0, height, 1):
        for j in range(0, width, 1):
            if angle_cell[i,j] < 0:
                ang = angle_cell[i,j] + PI
            if ang < PI/6:
                cell_list[0] += grad_cell[i,j]
            elif ang < PI/3:
                cell_list[1] += grad_cell[i,j]
            elif ang < PI/2:
                cell_list[2] += grad_cell[i,j]
            elif ang < PI*2/3:
                cell_list[3] += grad_cell[i,j]
            elif ang < PI*5/6:
                cell_list[4] += grad_cell[i,j]
            else:
                cell_list[5] += grad_cell[i,j]
    return cell_list

def all_cell_hist(grad_array, angle_array, stride):
    height, width = grad_array.shape
    new_h, new_w = height/stride, width/stride
    all_cell_list = []
    for i in range(0, height, stride):
        for j in range(0, width, stride):
            grad_cell = grad_array[i:i+stride, j:j+stride]
            angle_cell = angle_array[i:i+stride, j:j+stride]
            all_cell_list.append(single_cell_hist(grad_cell, angle_cell))
    return (new_h, new_w), all_cell_list

def block_his_map(cell_map, cell_map_size, block_size, stride):
    block_count = (np.array(cell_map_size) - np.array(block_size))/stride + 1
    all_block_feature = []
    for i in range(0, block_count[0],1):
        for j in range(0, block_count[1], 1):
            single_block_feature = []
            top_left = i*stride*cell_map_size[1] + j*stride
            for m in range(0, block_size[0],1):
                for n in range(0, block_size[1],1):    
                    t = cell_map[top_left + m*cell_map_size[1]+n]
                    single_block_feature.extend(t)
            all_block_feature.append(single_block_feature)
            single_block_feature = []
    return all_block_feature
    
    
if __name__ == "__main__":
    img, height, width = image_load("G:/local/target_detect/person.jpg")
    gamma(img)
    grad_array, angle_array = sobel_gradient(img)
    cell_map_count, cell_map = all_cell_hist(grad_array, angle_array, 5)
    all_block_feature = block_his_map(cell_map, cell_map_count, (10, 10), 5)
    
    print len(all_block_feature)
    plt.plot(all_block_feature[100], 'b-')
    plt.show()    

589
